In [ ]:
import scanpy as sc
import numpy as np

In [ ]:
import rpy2
import rpy2.robjects as robjects

In [ ]:
import anndata2ri
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

In [ ]:
from rpy2.robjects.packages import importr, data

In [ ]:
from openproblems.tasks.cell_cell_communication.metrics.odds_ratio import odds_ratio

#### Brain

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/allen_brain.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/allen_brain.h5ad")

In [ ]:
adata.uns["bench"]

In [ ]:
adata

In [ ]:
sc.pp.subsample(adata, n_obs=500)
adata = adata[:, 30000:30500].copy()

In [ ]:
adata

#### TNBC

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/brca_tnbc.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/brca_tnbc.h5ad")

### Test Data LOAD, norm!!!

##### BRAIN

In [ ]:
from openproblems.data.allen_brain_atlas import load_mouse_brain_atlas

In [ ]:
adata = load_mouse_brain_atlas(test=False)

In [ ]:
adata

##### TNBC

In [ ]:
from openproblems.data.tnbc_wu2021 import load_tnbc_data
adata = load_tnbc_data(test=False)
adata

In [ ]:
sc.pp.subsample(adata, n_obs=500)

In [ ]:
adata[:, adata.var.index.isin(set(adata.uns["bench"].ligand))].var.index

In [ ]:
# response ligands
lr = list(set(adata.uns["bench"].ligand))

# corresponding receptors
lr = lr + ["IFNLR1", "CD70", "NGFR", "CD53", "IL20RA",
           "IL20RB", "IL2RG", "SIGLEC6", "EGFR", "ITGB1",
           "ITGB3", "ACVRL1", "CXCR4", "SMAD3", "CAV1",
           "IL21R", "IL4R", "IL2RG", "IL13RA1", "IL13RA2"
          ]

In [ ]:
adata = adata[:, adata.var.index.isin(lr)]

Normalize internally in op

In [ ]:
from openproblems.tools.normalize import log_cpm

In [ ]:
adata = log_cpm(adata)

In [ ]:
adata.layers['logcounts'] = adata.layers['log_cpm']
del adata.layers['log_cpm']

In [ ]:
adata

#### TEST method + metric!!!!

### RUN LIANA

In [ ]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i adata -o adata
library(tidyverse)
library(liana)

method = "natmi"

adata@assays@data$counts <- as(as.matrix(adata@assays@data$logcounts), "sparseMatrix")
adata@assays@data$logcounts <- as(as.matrix(adata@assays@data$counts), "sparseMatrix")


# Here, we will convert LIANA's Consensus resource to murine symbols
op_resource <- select_resource("Consensus")[[1]]

# Check if the target organism is human
if(adata@metadata$target_organism!=9606){
    # Generate orthologous resource
    op_resource <- generate_homologs(op_resource = op_resource,
                                     target_organism = adata@metadata$target_organism)
}


# Run LIANA
adata@metadata$liana_res = liana_wrap(adata,
                                      resource = 'custom',# resource has to be set to 'custom' to work with external resources
                                      external_resource = op_resource, # provide orthologous resource
                                      method=method,
                                      expr_prop = 0.1
                                      
                        ) %>%
    # DEAL WITH THIS AT A PER-METHOD BASIS (in R again) ^ code above is shared for all methods
    mutate(score := .data[[liana:::.score_specs()[[method]]@method_score]]) %>%
    arrange(desc(score))

In [ ]:
# Merge liana's results with the truth
gt = adata.uns["bench"].merge(adata.uns["liana_res"], how='right')
adata.uns["truth"] = gt[gt['response'].notna()]

In [ ]:
adata.uns["truth"]

In [ ]:
odds_ratio(adata)